In [1]:
import numpy as np
import pickle
import json
import os
import sys
import copy
import sklearn.preprocessing
import models
from sklearn.neighbors import NearestNeighbors
import utils

In [2]:
DATA_DIR = "../../data/fb15k/"
DUMP_FILE = "../dumps/fb15k_distmult_dump_norm.pkl"
MODEL_TYPE = "distmult"
mining_dir="mined"

In [3]:
train_data = utils.read_data(os.path.join(DATA_DIR,"train.txt"))
# dev_data = read_data(os.path.join(DATA_DIR,"valid.txt"))
# test_data = read_data(os.path.join(DATA_DIR,"test.txt"))

In [4]:
dump=utils.load_pickle(DUMP_FILE)
dump.keys()

dict_keys(['tail_rel_type', 'entity_type', 'entity_to_id', 'head_rel_type', 'entity_real', 'rel_real', 'relation_to_id'])

In [5]:
model=models.TypedDM(DUMP_FILE)

In [9]:
mapped_train_data = utils.map_data(train_data,dump)
print(len(mapped_train_data))
# mapped_dev_data = map_data(dev_data)
# mapped_test_data = map_data(test_data)

483142


In [12]:
e1_e2_r,e2_e1_r=utils.get_ent_ent_rel(mapped_train_data)
print(len(e1_e2_r),len(e2_e1_r))

14834 14903


In [13]:
entity_count=len(dump['entity_to_id'])
print(entity_count)

14952


In [14]:
def hard_simi(lis1,lis2):
    s1=set(lis1)
    s2=set(lis2)
    return list(s1&s2)

In [15]:
def soft_simi(lis1,lis2,model):
    list_r1r2=[]
    
    for r1 in lis1:
        best_score=-1
        best_rel=-1
        for r2 in lis2:
            score=model.get_relation_similarity(r1,r2)
            if(score>best_score):
                best_score=score
                best_rel=r2
        if(best_rel!=-1):
            list_r1r2.append((r1,r2))
    
    return list_r1r2

In [ ]:
def explain_similarity_aux(e1,e2,model,flip,hard_match=True):
    d1=look_up[e1]
    d2=look_up[e2]
    relevant_tuple=[]
    
    for e in d1:
        if e not in d2:
            continue
        lis1=d1[e]
        lis2=d2[e]
        
        if(flag):
            relevant=hard_simi(lis1,lis2)
        else:
            relevant=soft_simi(lis1,lis2,model)
    
    for r1r2 in relevant:
        if flip:
            relevant_tuple.append(([e,r1r2[0],e1],[e,r1r2[1],e2]))
        else:
            relevant_tuple.append(([e1,r1r2[0],e],[e2,r1r2[1],e]))
    
    return relevant_tuple               